Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [10]:
pickle_file = 'data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [11]:
image_size = 28
num_labels = 10

# 注意这个函数修改了原始的dataset和labels，不能执行多次
def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32) # reshape -1表示维度不变
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    # 把一维的labels变成2维的，假设输入是[2, 0, 1]将其变为
    #[[0,0,1], -> 2
    # [1,0,0], -> 0
    # [0,1,0]] -> 1
    # 方法是先生成一个0,1,2,3,4..9的向量，形状是(10,)
    # label的形状是(10000,)，labels[:,None]的形状是(20000, 1)，它相当于reshape(20000, 1)，但它更快吧，因为用了slice操作
    # ==操作会让这两个参数broadcast，最后结果的形状是(20000, 10)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

(200000,)
(200000, 1)
(10000,)
(10000, 1)
(10000,)
(10000, 1)
Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [12]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  # 定义常量，对于tensorflow来说，这些数据是不可变的
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :]) # 只取最后的10000个训练数据
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  # 所有的参数都应该是变量，因为他们可以变化
  # 初始化weight，正态分布、均值0、标准差1，truncated表示超过两个标准差的数据将被丢弃
  weights = tf.Variable( # Variable居然是大写的。。。
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels])) # 全零的偏置
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  # 计算 y = XW +b
  logits = tf.matmul(tf_train_dataset, weights) + biases
  # softmax_cross_entropy_with_logits 执行 cross_entropy(softmax(logistis), y)，
  # 这里y就是train_labels（这个y不是一个向量，而是二维数组，见上面的转换步骤），
  # 其结果就是当前的W和b对每个训练数据的交叉熵
  # tf.reduce_mean 就是求当前W和b对所有训练数据的交叉熵的平均值，应用这个W和b整个数据集中所有数据与其正确分类偏差距离的均值
  # 这里并不是真正计算loss值，而只是定义其计算方法——tensor
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  # GradientDescentOptimizer是一个类，0.5是构造函数参数learning_rate的值
  # minimize是尝试最小化loss值，其运算过程中会更新所有Variable的值，这里只有W和b是Variable
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  # 用优化后的W和b分别对训练集、验证集和测试集计算分类可能性softmax，后续可以根据这个值计算分类器的准确率
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

# 上面所有都只定义了操作，而不是去执行

Let's run this computation and iterate:

In [14]:
num_steps = 801

def accuracy(predictions, labels):
  # np.argmax(predictions, 1)：对每个数据找到最大的预测可能性的值对应的索引值（即预测出的正确分类）
  # np.argmax(labels, 1)：对每个数据的正确分类
  # np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)：对于一条数据，如果预测出正确分类，返回true，否则false
  # 100 * (true or false) / 数据个数 = 当前分类器在整个数据集熵的准确率
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

# 用上面定义的graph启动一个会话
with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  # 初始化所有global variables，包括W和b
  tf.global_variables_initializer().run()
  print('Initialized')
  # 循环num_steps步
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    # session变量来自于with .... as session
    # session.run表示执行表达式或运算符，它执行第一个参数fetches中的每个图形元素，这些图形元素可以是：
    # 1）tf.Operation，比如tf.matmul，这里的optimizer、loss、train_prediction都是Operation
    # 2）tf.tensor，它表示Operation的一个输出，tensor定义了操作的输出，它同时可以作为另外一个Operation的输入，而多个Operation构成了整个Graph
    # 输出分别对应optimizer、loss、train_prediction的值，即使我们不需要optimizer的结果，但也将将它作为run的参数，因为这样才会更新W和b
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      # 每100步输出一次loss值和在训练集上的分类准确率
      print('Loss at step %d: %f' % (step, l))
      # 我们只取了一部分训练集进行计算，所以这里也只获取一部分训练集的标签进行比较。这里相当于在训练集上训练并预测，自我验证。
      print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      # 输出在验证集上的准确率，先需要对valid_prediction操作表达式执行eval方法，这样会用当前循环中更新后的W和b计算它。
      # eval方法会计算操作表达式本身以及作为其输入的其它操作
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  # 最后输出在测试集上的准确率
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 16.344929
Training accuracy: 12.8%
Validation accuracy: 15.4%
Loss at step 100: 2.276767
Training accuracy: 71.5%
Validation accuracy: 70.0%
Loss at step 200: 1.828829
Training accuracy: 74.3%
Validation accuracy: 72.4%
Loss at step 300: 1.585206
Training accuracy: 75.9%
Validation accuracy: 73.4%
Loss at step 400: 1.421627
Training accuracy: 77.0%
Validation accuracy: 74.0%
Loss at step 500: 1.301501
Training accuracy: 77.8%
Validation accuracy: 74.5%
Loss at step 600: 1.208143
Training accuracy: 78.4%
Validation accuracy: 74.7%
Loss at step 700: 1.132737
Training accuracy: 78.8%
Validation accuracy: 74.6%
Loss at step 800: 1.070133
Training accuracy: 79.2%
Validation accuracy: 74.7%
Test accuracy: 82.4%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [15]:
batch_size = 128 # 随机数据集的大小

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  # 上面的测试是把所有训练数据放到一个constant node中，而随机梯度下降算法中，将随机选中的一部分训练数据放到Placeholder node中
  # 随机选择的数据集和标签集都需要放到placeholder node中
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  # 验证集和测试集还放到constant node中
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.创建W和b
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # 创建损失函数（数据和标签都是来自placeholder节点中的数据和标签）
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [18]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  # 初始化所有的variables
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    # %防止最后的一段offset + batch_size超出数据集大小，这样可能会导致数据集中的最后一小段数据永远不会被选中参与训练
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    # 去offset 到 (offset + batch_size) 这一段数据作为minibatch，这个batch_data和batch_labels是真实的ndarray，而不是tf的operation
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    # 创建一个feed_dict，key是placeholder node的名字，value是真正的数据
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    # 把创建的feed_dict作为参数传递给session.run方法，它将覆盖同名的输入数据
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)) # 选中数据集mini_batch上的准确率，而不是整个训练集
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 20.221796
Minibatch accuracy: 9.4%
Validation accuracy: 12.2%
Minibatch loss at step 500: 1.284577
Minibatch accuracy: 77.3%
Validation accuracy: 76.0%
Minibatch loss at step 1000: 1.412993
Minibatch accuracy: 78.1%
Validation accuracy: 77.1%
Minibatch loss at step 1500: 0.803352
Minibatch accuracy: 82.8%
Validation accuracy: 77.0%
Minibatch loss at step 2000: 0.873147
Minibatch accuracy: 82.0%
Validation accuracy: 77.6%
Minibatch loss at step 2500: 0.933286
Minibatch accuracy: 78.9%
Validation accuracy: 78.7%
Minibatch loss at step 3000: 0.972521
Minibatch accuracy: 76.6%
Validation accuracy: 78.9%
Test accuracy: 86.5%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [25]:
batch_size = 128 # 随机数据集的大小
hidden_layer_size = 1024

def forward_logits(dataset, w1, b1, w2, b2):
  logits1 = tf.matmul(dataset, w1) + b1 # (batch_size, hidden_layer_size)
  relu = tf.nn.relu(logits1) # (batch_size, hidden_layer_size)
  logits2 = tf.matmul(relu, w2) + b2 # (hidden_layer_size, num_labels)
  return logits2

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  # 上面的测试是把所有训练数据放到一个constant node中，而随机梯度下降算法中，将随机选中的一部分训练数据放到Placeholder node中
  # 随机选择的数据集和标签集都需要放到placeholder node中
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  # 验证集和测试集还放到constant node中
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.创建W1和b1
  w1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  b1 = tf.Variable(tf.zeros([hidden_layer_size]))
  w2 = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # 创建损失函数（数据和标签都是来自placeholder节点中的数据和标签）
  logits2 = forward_logits(tf_train_dataset, w1, b1, w2, b2)
#   logits1 = tf.matmul(tf_train_dataset, weights1) + biases1 # (batch_size, hidden_layer_size)
#   relu = tf.nn.relu(logits1) # (batch_size, hidden_layer_size)
#   logits2 = tf.matmul(relu, weight2) + biases2 # (hidden_layer_size, num_labels)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits2)
  valid_prediction = tf.nn.softmax(forward_logits(tf_valid_dataset, w1, b1, w2, b2))
  test_prediction = tf.nn.softmax(forward_logits(tf_test_dataset, w1, b1, w2, b2))

In [26]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  # 初始化所有的variables
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    # %防止最后的一段offset + batch_size超出数据集大小，这样可能会导致数据集中的最后一小段数据永远不会被选中参与训练
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    # 去offset 到 (offset + batch_size) 这一段数据作为minibatch，这个batch_data和batch_labels是真实的ndarray，而不是tf的operation
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    # 创建一个feed_dict，key是placeholder node的名字，value是真正的数据
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    # 把创建的feed_dict作为参数传递给session.run方法，它将覆盖同名的输入数据
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)) # 选中数据集mini_batch上的准确率，而不是整个训练集
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 313.967041
Minibatch accuracy: 10.9%
Validation accuracy: 27.3%
Minibatch loss at step 500: 17.966501
Minibatch accuracy: 81.2%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 9.129010
Minibatch accuracy: 78.1%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 7.003434
Minibatch accuracy: 82.8%
Validation accuracy: 82.3%
Minibatch loss at step 2000: 2.227365
Minibatch accuracy: 86.7%
Validation accuracy: 82.1%
Minibatch loss at step 2500: 2.281806
Minibatch accuracy: 85.9%
Validation accuracy: 81.9%
Minibatch loss at step 3000: 1.754063
Minibatch accuracy: 80.5%
Validation accuracy: 82.5%
Test accuracy: 89.4%
